In [96]:
%load_ext Cython

In [229]:
import numpy as np
with open("data.txt", "r") as r:
    data = np.array([[char for char in l.replace("\n", "")] for l in r.readlines()])

data[data == "."] = "0"
data[data == "L"] = "1"
data = data.astype(int)

floor_map = data.copy()

In [253]:
%%cython -a

import numpy as np
cimport numpy as np

cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double part_1_cython(np.ndarray[np.int32_t, ndim=2] data, np.ndarray[np.int32_t, ndim=2] floor_map):
    cdef int max_i = len(data)
    cdef int max_j = len(data[0])
    cdef int i, j, eye
    cdef np.ndarray[np.int32_t, ndim=2] new_seatings
    cdef int p 
    while True:
        new_seatings = np.zeros((data.shape[0], data.shape[1]), dtype=int)
        for i in range(max_i):
            for j in range(max_j):
                if floor_map[i,j] == 1:
                    p = data[i,j]
                    d = data[np.max([0, i-1]):np.min([max_i, i+2]), np.max([0, j-1]):np.min([max_j, j+2])]
                    eye = d.sum() - p
        
                    if eye > 3 and p == 1:
                        new_seatings[i,j] = 0
                    elif eye == 0 and p == 0:
                        new_seatings[i,j] =  1
                    else: 
                        new_seatings[i,j] = p
                else:
                    new_seatings[i,j] = 0

        if (data == new_seatings).all():
            break
        data = new_seatings

    return new_seatings.sum()


In [248]:
# part 1
def part_1(data, floor_map):
    max_i, max_j = len(data), len(data[0])
    while True:
        new_seatings = np.zeros(data.shape)
        for i in range(len(data)):
            for j in range(len(data[0])):
                if floor_map[i,j] == 1:
                    p = data[i,j]
                
                    d = data[np.max([0, i-1]):np.min([max_i, i+2]), np.max([0,j-1]):np.min([max_j, j+2])]
                    eye = eye = d.sum() - p
        
                    if eye > 3 and p == 1:
                        new_seatings[i,j] = 0
                    elif eye == 0 and p == 0:
                        new_seatings[i,j] =  1
                    else: 
                        new_seatings[i,j] = p
                else:
                    new_seatings[i,j] = 0
        if (data == new_seatings).all():
            break
        data = new_seatings

    return np.nan_to_num(new_seatings, nan=0).sum()

In [249]:
%timeit part_1(data, floor_map)

25 s ± 579 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [252]:
%timeit part_1_cython(data, floor_map) 

1min 2s ± 5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [246]:
a = [0,1]
%timeit min(a), 
%timeit np.min(a)

215 ns ± 3.63 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
6.22 µs ± 68.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
